In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("mapdata_wildfires.csv")

In [ ]:
coordinates_and_dates = []

for _, row in df.iterrows():
    lat = row['incident_latitude']
    lon = row['incident_longitude']
    date_created = row['incident_dateonly_created']
    
    # Create a tuple and append it to the list
    coordinates_and_dates.append((lat, lon, date_created))

In [ ]:
import requests

def get_weather_data(api_key, lat, lon, date):
    base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history"
    
    params = {
        "aggregateHours": "24",
        "startDateTime": date,
        "endDateTime": date,
        "unitGroup": "us",
        "contentType": "csv",
        "dayStartTime": "0:0:00",
        "dayEndTime": "0:0:00",
        "locationMode": "singlelatlong",
        "key": api_key,
        "dataElements": "all",
        "locations": f"{lat},{lon}"
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Error fetching data: {response.status_code}")
        return None

api_key = "" 


# Create a new CSV file and write the results
with open("weather_data_wildfires_intermediate.csv", "w") as csvfile:
    header_written = False
    for lat, lon, date in coordinates_and_dates:
        weather_data = get_weather_data(api_key, lat, lon, date)
        if weather_data:
            lines = weather_data.strip().split("\n")
            if not header_written:
                # Write header with "Latitude", "Longitude", "Date" added to the front
                csvfile.write("Latitude,Longitude,Date," + lines[0] + "\n")
                header_written = True
            
            if len(lines) < 2:
                print(f"No data found for {lat}, {lon} on {date}. Will continue with the next record")
                csvfile.write(f"{lat},{lon},{date},None\n") # Can comment this line out if missing record should not be included in the CSV file
                continue
            
            data_line = lines[1]
            # Split the data line into fields and remove the first three fields (Address, Address Type, Resolved Address)
            data_fields = data_line.split(",")[3:]
            
            # Write the latitude, longitude, date, and the rest of the weather data to the CSV file
            csvfile.write(f"{lat},{lon},{date},{','.join(data_fields)}\n")

print("Weather data exported to weather_data_wildfires_intermediate.csv")

In [ ]:
# Fix the errors and remove irrelevant features from the weather_data_wildfires_intermediate dataset

weather_data_wildfires = pd.read_csv('weather_data_wildfires_intermediate.csv')

weather_data_wildfires = weather_data_wildfires.drop(['Address', 'Latitude.1', 'Longitude.1',
       'Name'], axis = 1)

weather_data_wildfires = weather_data_wildfires.iloc[:,:-2]

weather_data_wildfires.columns = ['Latitude', 'Longitude', 'Date',
       'Maximum Temperature', 'Minimum Temperature', 'Temperature',
       'Dew Point', 'Wind Chill', 'Heat Index', 'Precipitation',
       'Precipitation Cover', 'Snow', 'Snow Depth', 'Wind Speed', 'Wind Gust',
       'Wind Direction', 'Visibility', 'Cloud Cover', 'Relative Humidity',
       'Sea Level Pressure', 'Solar Radiation', 'Solar Energy', 'Weather Type',
       'Info', 'Conditions']

weather_data_wildfires.to_csv("weather_data_wildfires.csv", index = False)

print("Weather data exported to weather_data_wildfires.csv")